### Loading the Data

In [2]:
import os
from audio_image_pipeline import preprocess_to_hdf5

# absolute path because he somehow could not find the data otherwise idk 
# here we just have our folder that contains our audio data 
audio_folder = "/Users/koraygecimli/PycharmProjects/UDL_demo/wewantCNNs_project/data/nsynth-valid/audio"

# this is the path where we save our converted data
output_h5_path = "/Users/koraygecimli/PycharmProjects/UDL_demo/wewantCNNs_project/data/train_spectrograms_even.h5"

preprocess_to_hdf5(audio_folder, output_h5_path) # turns audio data into hdf5 data 

100%|██████████| 12678/12678 [01:01<00:00, 206.30it/s]

Preprocessed 12678 files to /Users/koraygecimli/PycharmProjects/UDL_demo/wewantCNNs_project/data/train_spectrograms_even.h5


### Training

In [3]:
import torch
from torch.utils.data import DataLoader
from datasets import HDF5SpectrogramDataset
from nf_training_script import train_model  

In [4]:
import torch
from torch.utils.data import DataLoader
from datasets import HDF5SpectrogramDataset
from nf_training_script import train_model  

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

train_data = HDF5SpectrogramDataset('/Users/koraygecimli/PycharmProjects/UDL_demo/wewantCNNs_project/data/train_spectrograms_even.h5')

train_loader = DataLoader( # again inspired by vae model 
    train_data,
    batch_size=32,  
    shuffle=True,
    pin_memory=True,
    num_workers=0,
    persistent_workers=False,
    prefetch_factor=None,
)

input_dim = train_data[0].numel()
trained_model = train_model(train_loader, input_dim=input_dim)

Using device: cpu


/Users/koraygecimli/PycharmProjects/UDL_demo/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/20, Loss: -86942.2199
Checkpoint saved at epoch 1
Epoch 2/20, Loss: -89821.6244
Checkpoint saved at epoch 2
Epoch 3/20, Loss: -89835.1581
Checkpoint saved at epoch 3
Epoch 4/20, Loss: -89839.8678
Checkpoint saved at epoch 4
Epoch 5/20, Loss: -89841.4452
Checkpoint saved at epoch 5
Epoch 6/20, Loss: -89842.2912
Checkpoint saved at epoch 6
Epoch 7/20, Loss: -89843.1520
Checkpoint saved at epoch 7
Epoch 8/20, Loss: -89843.9581
Checkpoint saved at epoch 8
Epoch 9/20, Loss: -89844.7354
Checkpoint saved at epoch 9
Epoch 10/20, Loss: -89845.5572
Checkpoint saved at epoch 10
Epoch 11/20, Loss: -89846.3446
Checkpoint saved at epoch 11
Epoch 12/20, Loss: -89847.1391
Checkpoint saved at epoch 12
Epoch 13/20, Loss: -89847.9359
Checkpoint saved at epoch 13
Epoch 14/20, Loss: -89848.5683
Checkpoint saved at epoch 14
Epoch 15/20, Loss: -89848.6246
Checkpoint saved at epoch 15
Epoch 16/20, Loss: -89848.6289
Checkpoint saved at epoch 16
Epoch 17/20, Loss: -89848.6301
Checkpoint saved at epoch 1

In [3]:
from inference import load_nf_model
from inference import invert_flow

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = load_nf_model(device)
z_dim = model.output_layer.out_features // 2  # oder einfach input_dim aus dem Checkpoint
z = torch.randn(2, z_dim).to(device)  # 2 Samples aus latenten Raum
x_samples = invert_flow(model, z, device)
print(x_samples.shape)
print(x_samples)

torch.Size([2, 22016])
tensor([[ 125.5509,  113.5383,   92.0724,  ...,  166.9994,  -20.2034,
         -141.4516],
        [-154.3873,  153.4724, -106.3017,  ..., -273.2191,  159.3956,
         -278.9794]], device='mps:0')
